In [1]:
import os
import h5py
import numpy as np
import pandas as pd

from skimage.io import imread as imread_png
from skimage.measure import label, regionprops_table, regionprops
from tifffile import imread,imsave


In [2]:
path_frames_dir = './data_frames'
path_segmentation_dir = './segmentation_frames'
path_singles_dir = './singles_ch01'
info_file_path = './otsu_selPlanes.csv'

# properties of cells to be calculated
properties = ['label', 'area','centroid','bbox','major_axis_length','minor_axis_length',
              'local_centroid','filled_image','intensity_image'] 

# size of images of individual cells
im_size = 80
# bit depth for images of individual cells
bit8 = True

In [3]:
# find all masks of the cells
file_list = [x for x in os.listdir(path_segmentation_dir) if 'cells' in x]
len(file_list)

56

## Calculate features of cells and count foci

In [4]:
cell_data_list = []

for im_file in file_list:

    # open images
    cells_mask = imread_png(os.path.join(path_segmentation_dir,im_file))
    foci_mask = imread_png(os.path.join(path_segmentation_dir,im_file.replace('cells','foci')))

    frame_name = '_'.join(im_file.split('_')[:-2]) + '_ch01.tiff'
    ch1_im = imread(os.path.join(path_frames_dir,frame_name))

    # calculate properties of regions
    cell_data = regionprops_table(cells_mask,intensity_image = ch1_im, properties=properties)
    cell_data = pd.DataFrame(cell_data)

    # add foci info
    for ind,cell in cell_data.iterrows():

        # cut with bbox
        foci_bbox = foci_mask[cell['bbox-0']:cell['bbox-2'],cell['bbox-1']:cell['bbox-3']]
    
        # clean the area outside of a cell
        foci_bbox[~(cell.filled_image)]=0
    
        # label and regionprops
        foci_labels = label(foci_bbox)
        props = regionprops(foci_labels)
    
        # store the info about foci number
        cell_data.loc[ind,'foci_number']=len(props)

    # add info of these measurements
    cell_data['im_frame'] = '_'.join(im_file.split('_')[:-2])

    cell_data_list.append(cell_data)

cell_data_all = pd.concat(cell_data_list,ignore_index=True)

# names of individual cells
cell_data_all['cell_name'] = [(x+'_'+str(y).zfill(4)) for x,y in zip(cell_data_all.im_frame,cell_data_all.label)]

# calculate volume
cell_data_all['volume'] = (4*np.pi*(cell_data_all.major_axis_length/2)*(cell_data_all.minor_axis_length/2)**2)/3

cell_data_all.to_pickle('./cell_data.pkl')

## Cut images of single cells

In [6]:
for ind,cell in cell_data_all.iterrows():
    
    # create empty 
    cell_im = np.zeros([im_size,im_size])
    
    cell_insert = cell.intensity_image
    
    row_start = int(im_size/2-(cell_insert.shape[0]/2))
    column_start = int(im_size/2-int(cell_insert.shape[1]/2))
    cell_im[row_start:row_start+cell_insert.shape[0],column_start:column_start+cell_insert.shape[1]] = cell_insert
    
    cell_name = cell.cell_name + '.tiff'
    
    if bit8 == True:
        
        # rescale to 8 bit
        resc = np.quantile(cell_im,0.999)
        cell_im = cell_im/resc*255
        cell_im[cell_im > 255] = 255

        imsave(os.path.join(path_singles_dir,cell_name),cell_im.astype('uint8'))
    else:
        imsave(os.path.join(path_singles_dir,cell_name),cell_im)